In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
import re
from matplotlib import pyplot as plt

In [2]:
dataset = pd.read_csv('news_summary.csv')#,encoding="ISO-8859-1")
dataset.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",Virgin' now corrected to 'Unmarried' in IGIMS'...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [3]:
dataset = dataset.dropna(how='any',axis=0) 
x = dataset['ctext']
y = dataset['text']
headline = dataset['headlines']
del dataset

In [4]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [6]:
x = [preprocess_sentence(w) for w in x]
y = [preprocess_sentence(w) for w in y]
headline = [preprocess_sentence(w) for w in headline]
# x_train, inp_lang = tokenize(x)
# y_train, targ_lang = tokenize(y)
# headline_train , headline_lang = tokenize(headline)
# print("Input Tensor Shape :" , x_train.shape , "Target Tensor Shape :" , y_train.shape)
print("Preprocessed Text : \n" , x[10])
# print("Tokenized Text : \n" , x_train[10])
print("Headline Text : \n" , headline[10])
# print("Headline Tokenized : \n" , headline_train[10])

Preprocessed Text : 
 <start> the food safety and standards authority of india fssai is in the process of creating a network of food banking partners to collect and distribute leftover food from large parties and weddings to the hungry . a notification to create a separate category of food business operators fbos , who will be licensed to deal only with leftover food , has been drafted to ensure the quality of food . ? we are looking at partnering with ngos or organisations that collect , store and distribute surplus food to ensure they maintain certain hygiene and health standards when handling food , ? said pawan agarwal , ceo of fssai . ? tonnes of food is wasted annually . we are looking at creating a mechanism through which food can be collected from restaurants , weddings , large scale parties , ? says pawan agarwal , ? all food , whether it is paid for or distributed free , must meet the country ? s food safety and hygiene standards , ? he said . the organisations in the busines

In [7]:
EMBEDDING_DIMENSION= 50
# data_directory = "/glove"
# glove_weights_file_path = "/glove/glove.6B.50d.txt"

# PAD_TOKEN = 0
word2idx= [] 
idx2word= []
weights = []
# f = open("glove_weights_file_path","r")
counts =[]
count =0
with open('glove.6B.50d.txt',encoding='utf-8') as file: 
    for index, line in enumerate(file): 
        values = line.split() # Word and weights separated by space 
        word = (values[0]) # Word is first symbol on each line 
        word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word 
        word2idx.append(word) # PAD is our zeroth index so shift by one 
        weights.append(word_weights)
        counts.append(index)
n_vec = index+1
# n_vec = i + 1
hidden_dim = len(line.split(' ')) - 1

vecs = np.zeros((n_vec, hidden_dim), dtype=np.float32)

with open('glove.6B.50d.txt', encoding = 'utf-8') as f:
    for i, line in enumerate(f):
        vecs[i] = np.array([float(n) for n in line.split(' ')[1:]], dtype=np.float32)

average_vec = np.mean(vecs, axis=0) #unknown vector
print("glove vctor loaded")

glove vctor loaded


In [8]:
print(word2idx[0])
print(weights[0])

the
[ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [9]:
word2idx.append('UNK')
weights.append(average_vec)
print(weights[word2idx.index('UNK')])

[-0.12920076 -0.28866628 -0.01224866 -0.05676644 -0.20210965 -0.08389011
  0.33359843  0.16045167  0.03867431  0.17833012  0.04696583 -0.00285802
  0.29099807  0.04613704 -0.20923874 -0.06613114 -0.06822549  0.07665912
  0.3134014   0.17848536 -0.1225775  -0.09916984 -0.07495987  0.06413227
  0.14441176  0.60894334  0.17463093  0.05335403 -0.01273871  0.03474107
 -0.8123879  -0.04688699  0.20193407  0.2031118  -0.03935686  0.06967544
 -0.01553638 -0.03405238 -0.06528071  0.12250231  0.13991883 -0.17446303
 -0.08011883  0.0849521  -0.01041659 -0.13705009  0.20127155  0.10069408
  0.00653003  0.01685157]


In [19]:
# Headline Embedding
h_features ={}
h_list=[]
h_features['headline_word_indices'] = nltk.word_tokenize(headline[1])
for word in h_features['headline_word_indices']:
    if word in word2idx:
        h_features['headline_word_indices']=weights[word2idx.index(word)]
    else:
        h_features['headline_word_indices']=weights[word2idx.index('UNK')]
    h_list.append(h_features['headline_word_indices'])

h_list = tf.convert_to_tensor(h_list)
h_list = tf.reshape(h_list,(-1,h_list.shape[0],h_list.shape[1]))
print(h_list.shape)

(1, 16, 50)


In [13]:
#document Embedding
features = {}
d_list=[]
features['word_indices'] = nltk.word_tokenize(x[1]) #embedding on the document
# features['word_indices'] = [weights[word2idx.index(word)] for word in features['word_indices']]
for word in features['word_indices']:
    if word in word2idx:
        features['word_indices'] = [weights[word2idx.index(word)]]
    else:
        features['word_indices'] =[weights[word2idx.index('UNK')]]
    d_list.append(features['word_indices'])

d_list = tf.convert_to_tensor(d_list)
d_list = tf.reshape(d_list,(1,d_list.shape[0],d_list.shape[2]))
print(d_list.shape)

(1, 460, 50)


In [14]:
class Encoder(tf.keras.Model):
    '''
    The code of encoder is used for text summarization. The model needs an initial_state which is the headline of the article.
    The input is supposed to be padded sequence for initial state as well as for the input. The model cuurently implements
    GRU for the encoder sequence as there is some bug with using lstm with bidirectional. Bidirectional was also tried with 
    GRU but same bug occured.
    
    The initialization is based on docuemnt context vector paper referenced from 
    https://arxiv.org/pdf/1807.08000.pdf paper named 
    Abstractive and Extractive Text Summarization using DocumentContext Vector and Recurrent Neural Networks.
    
    However the logic is modeified. The model takes the preprocessed tokenized padded sequence as input for initials state
    and the then it is passed through the embedded vector and the size of the output vector after embedding is (batch_size , time , embedding size)
    after this the model is averaged about the time axis and gives the output as (batch_size , embedding size).
    This tensor is then passed as input to dence vector and is used to give the output size of (batch_zie , lgur_hidden_units)
    '''
    def __init__(self,units = 16,activation='tanh',recurrent_activation='sigmoid',return_state=True,return_sequence=True , vocab_size=30000 , embedding_dim=30 , batch_size=2):
        super (Encoder , self).__init__()
        self.units = units
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.return_state = return_state
        self.return_sequence = return_sequence
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#         self.lstm = tf.keras.layers.LSTM(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)
        self.gru = tf.keras.layers.GRU(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)
#         self.bidirectional = tf.keras.layers.Bidirectional(self.gru)
        self.dense = tf.keras.layers.Dense(units=self.units , input_shape = (self.embedding_dim,))
    
    def call(self,x,headline):
#         x = self.embedding(x)
#         headline = self.embedding(headline)
        headline = tf.reduce_mean(headline,axis=1)
        headline  = self.dense(headline)
        output , state = self.gru(x,initial_state=headline)
        return output , state

In [15]:
encoder = Encoder()

In [17]:
output , state = encoder(d_list , h_list)
print(output.shape)
print(state.shape)

(1, 460, 16)
(1, 16)


In [20]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [21]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(state, output)
print(attention_result.shape)
print(attention_weights.shape)

(1, 16)
(1, 460, 1)


In [25]:
class Decoder(tf.keras.Model):
    
    def __init__(self, units = 16,activation='tanh',recurrent_activation='sigmoid',return_state=True,return_sequence=False , vocab_size=40000 , embedding_dim=30 , batch_size=2):
        super(Decoder, self).__init__()
        self.units = units
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.return_state = return_state
        self.return_sequence = return_sequence
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#         self.gru = tf.keras.layers.GRU(self.dec_units,
#                                        return_sequences=True,
#                                        return_state=True,
#                                        recurrent_initializer='glorot_uniform')
        self.gru = tf.keras.layers.GRU(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)

        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.units)

    def call(self, x, hidden, output):
    # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
#         x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[1]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [26]:
decoder = Decoder()

sample_decoder_output, _, _ = decoder(tf.random.uniform((1, 1,50)),state,output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (1, 40000)


In [27]:
def evaluate(text,headline):
    inputs = tf.convert_to_tensor(text)
    h = tf.convert_to_tensor(headline)
    result = ''

    enc_out, enc_hidden = encoder(inputs, h)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims(tf.expand_dims(tf.convert_to_tensor(weights[word2idx.index('start')]), 0),0)
    for t in range(100):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += word2idx[predicted_id] + ' '

        if word2idx[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims(tf.expand_dims(tf.convert_to_tensor(weights[predicted_id]), 0),0)

    return result

In [29]:
def translate(text,headline):
    result = evaluate(text,headline)
    print('Predicted translation: {}'.format(result))

In [30]:
translate(d_list,h_list)

Predicted translation: mclean reflective rouse 51st vestiges ascot secessionist dir wildly chagrin haider affirmed 7000 iranian afghanistan schott gainer sarin redress hysterical hercegovina slovene confessing wahl quotations painless zoologist nationalist fatality cra dur sensual fredrik erodes coughing lived outsiders trailed farrar minimized seifert rafting hp 10:30 lak midsize 7000 iranian afghanistan schott gainer sarin redress hysterical hercegovina slovene confessing wahl quotations painless zoologist nationalist fatality cra dur sensual fredrik erodes coughing lived outsiders trailed farrar minimized seifert rafting hp 10:30 lak midsize 7000 iranian afghanistan schott gainer sarin redress hysterical hercegovina slovene confessing wahl quotations painless zoologist nationalist fatality cra dur sensual 


In [40]:
#document Embedding
features = {}
d_list=[]
features['word_indices'] = nltk.word_tokenize(x[5]) #embedding on the document
# features['word_indices'] = [weights[word2idx.index(word)] for word in features['word_indices']]
for word in features['word_indices']:
    if word in word2idx:
        features['word_indices'] = [weights[word2idx.index(word)]]
    else:
        features['word_indices'] =[weights[word2idx.index('UNK')]]
    d_list.append(features['word_indices'])

d_list = tf.convert_to_tensor(d_list)
d_list = tf.reshape(d_list,(1,d_list.shape[0],d_list.shape[2]))
print(d_list.shape)

(1, 457, 50)


In [41]:
# Headline Embedding
h_features ={}
h_list=[]
h_features['headline_word_indices'] = nltk.word_tokenize(headline[5])
for word in h_features['headline_word_indices']:
    if word in word2idx:
        h_features['headline_word_indices']=weights[word2idx.index(word)]
    else:
        h_features['headline_word_indices']=weights[word2idx.index('UNK')] 
    h_list.append(h_features['headline_word_indices'])

h_list = tf.convert_to_tensor(h_list)
h_list = tf.reshape(h_list,(-1,h_list.shape[0],h_list.shape[1]))
print(h_list.shape)

(1, 18, 50)


In [42]:
translate(d_list,h_list)

Predicted translation: stowe hardest zoologist nationalist fatality cra dur sensual fredrik rm1 esther bauer 2005-2006 struggles fiore showed xfdws 1941 540 renovated sauce imaginative 29 soloists attila cognitive veered co-written skye pvt. bona asad scandal madre mart blended hilary petroleum fermentation coyne ezra painless tammy cnpc pies metacritic bailing riot 47-42-80-44 díaz fascists salespeople bestselling shortly criticizes recharge illawarra connor mounted existing bronfman violent appointments olivetti violent getter françois phrases acquisitions holmgren devise tribesmen competitors five-year combo customized folks mohan tethered neverland wahn ... handheld harp gainer sarin redress qualms computed reiterate microscope luzon positively cline nears stabilisation adviser gail 500m organises 
